In [2]:
from pytorch_pretrained_bert import OpenAIGPTDoubleHeadsModel, OpenAIGPTTokenizer

In [3]:
model = OpenAIGPTDoubleHeadsModel.from_pretrained('openai-gpt')
tokenizer = OpenAIGPTTokenizer.from_pretrained('openai-gpt')

100%|██████████| 458495/458495 [00:00<00:00, 829414.60B/s]
ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.
